Predicting total money spent on treating different cancers based on initial costs and patient demographics

In [1]:
# https://www.kaggle.com/rishidamarla/costs-for-cancer-treatment
# https://data.world/xprizeai-health/expenditures-for-cancer-care

In [2]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline


In [4]:
df = pd.read_csv('DowloadableDataFull_2011.01.12.csv')
df.head(5)

,Cost of Cancer Care by Phase of Care,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Costs ($) per year in millions of 2010 dollars,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,See http://costprojections.cancer.gov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cancer Site,Year,Sex,Age,Incidence and Survival Assumptions,Annual Cost Increase (applied to initial and l...,Total Costs,Initial Year After Diagnosis Cost,Continuing Phase Cost,Last Year of Life Cost
3,AllSites,2010,Both sexes,All ages,"Incidence, Survival at constant rate",0%,124565.6,40463.5,46642.8,37459.2
4,AllSites,2010,Both sexes,All ages,"Incidence follows recent trend, Survival constant",0%,122420.8,38552.7,46671.9,37196.3


#### Cleaning

In [5]:
# clearing unnecessary rows
df = df.iloc[2:]

# setting the column names
df.columns = df.iloc[0]

# dropping the extra row
df = df.drop(df.index[0])

# resetting the index to 0
df = df.reset_index(drop=True)

In [6]:
df.head(5)

2,Cancer Site,Year,Sex,Age,Incidence and Survival Assumptions,Annual Cost Increase (applied to initial and last phases),Total Costs,Initial Year After Diagnosis Cost,Continuing Phase Cost,Last Year of Life Cost
0,AllSites,2010,Both sexes,All ages,"Incidence, Survival at constant rate",0%,124565.6,40463.5,46642.8,37459.2
1,AllSites,2010,Both sexes,All ages,"Incidence follows recent trend, Survival constant",0%,122420.8,38552.7,46671.9,37196.3
2,AllSites,2010,Both sexes,All ages,"Survival follows recent trend, Incidence constant",0%,125397.7,40463.5,47136.3,37797.9
3,AllSites,2010,Both sexes,All ages,"Incidence, Survival follow recent trends",0%,123236.3,38552.7,47155.7,37527.8
4,AllSites,2010,Both sexes,All ages,"Incidence, Survival follow recent trends",2%,123236.3,38552.7,47155.7,37527.8


In [7]:
df.sample(10)

2,Cancer Site,Year,Sex,Age,Incidence and Survival Assumptions,Annual Cost Increase (applied to initial and last phases),Total Costs,Initial Year After Diagnosis Cost,Continuing Phase Cost,Last Year of Life Cost
304,Melanoma,2012,Both sexes,All ages,"Incidence, Survival follow recent trends",2%,2735.8,632.4,1719.2,384.2
109,Other,2010,Both sexes,All ages,"Incidence follows recent trend, Survival constant",0%,20599.0,4430.3,10925.6,5243.1
220,Uterus,2011,Both sexes,All ages,"Incidence, Survival follow recent trends",2%,2598.4,1036.9,787.0,774.4
522,Lymphoma,2014,Both sexes,All ages,"Incidence, Survival at constant rate",0%,13375.1,3372.5,5418.7,4583.9
1078,Kidney,2019,Both sexes,All ages,"Incidence, Survival follow recent trends",2%,6611.9,2124.5,2503.0,1984.4
744,Lung,2016,Both sexes,All ages,"Incidence, Survival at constant rate",0%,13648.0,5926.7,2076.2,5645.0
27,Cervix,2010,Females,All ages,"Incidence, Survival follow recent trends",0%,1394.4,386.0,355.7,652.6
979,Lymphoma,2018,Both sexes,All ages,"Incidence follows recent trend, Survival constant",0%,14415.1,3488.6,5891.4,5035.1
285,Leukemia,2012,Both sexes,All ages,"Incidence, Survival follow recent trends",0%,5631.6,781.3,2203.2,2647.1
1000,Pancreas,2018,Both sexes,All ages,"Incidence, Survival follow recent trends",2%,3413.0,1938.7,200.1,1274.2


In [8]:
df.isna().sum()

2
Cancer Site                                                  0
Year                                                         0
Sex                                                          0
Age                                                          0
Incidence and Survival Assumptions                           0
Annual Cost Increase (applied to initial and last phases)    0
Total Costs                                                  0
Initial Year After Diagnosis Cost                            0
Continuing Phase Cost                                        0
Last Year of Life Cost                                       0
dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1254 entries, 0 to 1253
Data columns (total 10 columns):
 #   Column                                                     Non-Null Count  Dtype 
---  ------                                                     --------------  ----- 
 0   Cancer Site                                                1254 non-null   object
 1   Year                                                       1254 non-null   object
 2   Sex                                                        1254 non-null   object
 3   Age                                                        1254 non-null   object
 4   Incidence and Survival Assumptions                         1254 non-null   object
 5   Annual Cost Increase (applied to initial and last phases)  1254 non-null   object
 6   Total Costs                                                1254 non-null   object
 7   Initial Year After Diagnosis Cost                          1254 non-null   object
 8   Continuing Phase C

In [10]:
df.iloc[:, [0,1,2,3,4]]

2,Cancer Site,Year,Sex,Age,Incidence and Survival Assumptions
0,AllSites,2010,Both sexes,All ages,"Incidence, Survival at constant rate"
1,AllSites,2010,Both sexes,All ages,"Incidence follows recent trend, Survival constant"
2,AllSites,2010,Both sexes,All ages,"Survival follows recent trend, Incidence constant"
3,AllSites,2010,Both sexes,All ages,"Incidence, Survival follow recent trends"
4,AllSites,2010,Both sexes,All ages,"Incidence, Survival follow recent trends"
...,...,...,...,...,...
1249,Other,2020,Both sexes,All ages,"Incidence follows recent trend, Survival constant"
1250,Other,2020,Both sexes,All ages,"Survival follows recent trend, Incidence constant"
1251,Other,2020,Both sexes,All ages,"Incidence, Survival follow recent trends"
1252,Other,2020,Both sexes,All ages,"Incidence, Survival follow recent trends"


In [11]:
cats = list(df.columns[0:5])
cats

['Cancer Site', 'Year', 'Sex', 'Age', 'Incidence and Survival Assumptions']

In [12]:
nums = list(df.columns[5:])
nums

['Annual Cost Increase (applied to initial and last phases)',
 'Total Costs',
 'Initial Year After Diagnosis Cost',
 'Continuing Phase Cost',
 'Last Year of Life Cost']

In [13]:
for i in cats:
    print(df[i].value_counts())

Stomach       66
Pancreas      66
Esophagus     66
Other         66
Leukemia      66
Kidney        66
Brain         66
Colorectal    66
Cervix        66
Melanoma      66
Uterus        66
Head_Neck     66
Lymphoma      66
Bladder       66
Breast        66
Ovary         66
Prostate      66
AllSites      66
Lung          66
Name: Cancer Site, dtype: int64
2013    114
2016    114
2012    114
2020    114
2011    114
2015    114
2018    114
2019    114
2014    114
2010    114
2017    114
Name: Year, dtype: int64
Both sexes    990
Females       198
Males          66
Name: Sex, dtype: int64
All ages    1254
Name: Age, dtype: int64
Incidence, Survival follow recent trends             627
Survival follows recent trend, Incidence constant    209
Incidence, Survival at constant rate                 209
Incidence follows recent trend, Survival constant    209
Name: Incidence and Survival Assumptions, dtype: int64


In [14]:
df.shape

(1254, 10)

In [15]:
print(df['Annual Cost Increase (applied to initial and last phases)'].value_counts())
print(df.loc[10, 'Annual Cost Increase (applied to initial and last phases)'][:-1])
for i in range(len(df)):
    df.loc[i, 'Annual Cost Increase (applied to initial and last phases)'] = df.loc[i, 'Annual Cost Increase (applied to initial and last phases)'][:-1]

0%    836
2%    209
5%    209
Name: Annual Cost Increase (applied to initial and last phases), dtype: int64
2


In [16]:
print(df.loc[10, 'Annual Cost Increase (applied to initial and last phases)'])
print(df['Annual Cost Increase (applied to initial and last phases)'].value_counts())

2
0    836
5    209
2    209
Name: Annual Cost Increase (applied to initial and last phases), dtype: int64


In [17]:
for i in nums:
    df[i] = df[i].astype(float)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1254 entries, 0 to 1253
Data columns (total 10 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Cancer Site                                                1254 non-null   object 
 1   Year                                                       1254 non-null   object 
 2   Sex                                                        1254 non-null   object 
 3   Age                                                        1254 non-null   object 
 4   Incidence and Survival Assumptions                         1254 non-null   object 
 5   Annual Cost Increase (applied to initial and last phases)  1254 non-null   float64
 6   Total Costs                                                1254 non-null   float64
 7   Initial Year After Diagnosis Cost                          1254 non-null   float64
 8   Continui

In [19]:
df.iloc[:, 5:].describe()

2,Annual Cost Increase (applied to initial and last phases),Total Costs,Initial Year After Diagnosis Cost,Continuing Phase Cost,Last Year of Life Cost
count,1254.000000,1254.000000,1254.000000,1254.000000,1254.000000
mean,1.166667,15221.008852,4677.505742,5764.655024,4778.848804
std,1.864133,31489.503603,9703.258460,12094.699099,9930.281985
min,0.000000,1204.600000,293.000000,98.900000,330.600000
25%,0.000000,2866.625000,939.175000,846.875000,908.925000
50%,0.000000,5107.350000,1377.150000,1940.350000,1755.200000
75%,2.000000,14178.925000,4883.050000,5187.200000,4465.275000
max,5.000000,206587.100000,68858.000000,66099.700000,74159.700000


In [20]:
df.iloc[:, 5:].corr()

2,Annual Cost Increase (applied to initial and last phases),Total Costs,Initial Year After Diagnosis Cost,Continuing Phase Cost,Last Year of Life Cost
2,,,,,
Annual Cost Increase (applied to initial and last phases),1.000000,0.026826,0.039405,0.002160,0.043933
Total Costs,0.026826,1.000000,0.993141,0.991058,0.993552
Initial Year After Diagnosis Cost,0.039405,0.993141,1.000000,0.973080,0.986996
Continuing Phase Cost,0.002160,0.991058,0.973080,1.000000,0.973907
Last Year of Life Cost,0.043933,0.993552,0.986996,0.973907,1.000000


In [21]:
df['Age'].value_counts()

All ages    1254
Name: Age, dtype: int64

In [22]:
df.drop('Age', axis=1, inplace=True)

In [23]:
df.head(4)

2,Cancer Site,Year,Sex,Incidence and Survival Assumptions,Annual Cost Increase (applied to initial and last phases),Total Costs,Initial Year After Diagnosis Cost,Continuing Phase Cost,Last Year of Life Cost
0,AllSites,2010,Both sexes,"Incidence, Survival at constant rate",0.0,124565.6,40463.5,46642.8,37459.2
1,AllSites,2010,Both sexes,"Incidence follows recent trend, Survival constant",0.0,122420.8,38552.7,46671.9,37196.3
2,AllSites,2010,Both sexes,"Survival follows recent trend, Incidence constant",0.0,125397.7,40463.5,47136.3,37797.9
3,AllSites,2010,Both sexes,"Incidence, Survival follow recent trends",0.0,123236.3,38552.7,47155.7,37527.8


#### Exploring the Data

In [ ]:
# total costs average
# initial

In [ ]:
# total costs average grouped by year
# initial

In [ ]:
# total costs average grouped by site
# initial

In [ ]:
# total costs average grouped by annual cost increase
# initial

In [ ]:
# total costs average grouped by sex
# initial

#### Plots

In [ ]:
# freq plot of cancer sites

In [ ]:
# freq plot of year

In [ ]:
# total costs (x) and annual cost increase (y) dot chart

In [ ]:
# initial year costs (x) by gender dot chart

In [ ]:
# total costs by gender

#### Feature Transformation and Engineering

In [ ]:
# split dataset into cols and label

In [ ]:
# drop last two columns and explain why

In [ ]:
# change year to either 0-10 numeric or 0-10 cat

In [ ]:
# change annual cost increase to cat

In [ ]:
# normalize numeric cols

In [ ]:
# ohe cat cols